# How this project was created

## Create a directory for the prom-keycloak-proxy project

In [ ]:
install -d ~/.local/src/prom-keycloak-proxy
cd ~/.local/src/prom-keycloak-proxy
echo DONE

## Initialize the go project

In [ ]:
go mod init github.com/OCP-on-NERC/prom-keycloak-proxy
echo DONE

## Get go dependencies

In [ ]:
go get github.com/Nerzal/gocloak/v13
go get github.com/gorilla/mux
echo DONE

## Develop the code

Create an errors package and HTTP errors

In [ ]:
install -d src/errors/

cat <<'EOF' > src/errors/httpError.go
// Thanks to okemechris on GitHub for the sample code. 
// See: https://github.com/okemechris/simplego-api/tree/main

package errors

type HttpError struct {
    Code int `json:"code"`
    Error string `json:"error"`
    Message string `json:"message"`
}

func UnauthorizedError()  HttpError{
    return HttpError{
        401,
        "Unauthorized",
        "You are not authorized to access this resource",
    }

}

func NotFoundError()  *HttpError{
    return &HttpError{
        404,
        "Not found",
        "The requested resource was not found",
    }

}

func DataAccessLayerError(message string)  *HttpError{
    return &HttpError{
        400,
        "Data access error",
        message,
    }

}

func BadRequestError(message string)  *HttpError{
    return &HttpError{
        400,
        "Bad Request",
        message,
    }

}
EOF
echo DONE

Create a Keycloak Authentication and Authorization service

In [ ]:
install -d src/services/

cat <<'EOF' > src/services/authService.go
// Thanks to okemechris on GitHub for the sample code. 
// See: https://github.com/okemechris/simplego-api/tree/main

package services

import (
    "os"
    "encoding/json"
    "net/http"
    "github.com/OCP-on-NERC/prom-keycloak-proxy/src/errors"
    "strings"
    "strconv"
    "crypto/tls"
    "fmt"
    "context"

    "github.com/Nerzal/gocloak/v13"
    _ "github.com/gorilla/mux"
)

type LoginResponse struct {
    AccessToken string `json:"access_token"`
    Title       string `json:"Title"`
    Description string `json:"Description"`
}

var (
    clientId             = os.Getenv("AUTH_CLIENT_ID")
    clientSecret         = os.Getenv("AUTH_CLIENT_SECRET")
    realm                = os.Getenv("AUTH_REALM")
    auth_base_url        = os.Getenv("AUTH_BASE_URL")
    auth_skip_verify, _  = strconv.ParseBool(os.Getenv("AUTH_SKIP_VERIFY"))
)

func InitializeOauthServer() *gocloak.GoCloak {
    client := gocloak.NewClient(auth_base_url)
    if auth_skip_verify {
        restyClient := client.RestyClient()
        restyClient.SetTLSClientConfig(&tls.Config{ InsecureSkipVerify: true })
    }
    return client
}

func Protect(client *gocloak.GoCloak, next http.Handler) http.Handler {

    return http.HandlerFunc(func(w http.ResponseWriter, r *http.Request) {

        authHeader := r.Header.Get("Authorization")

        if len(authHeader) < 1 {
            w.WriteHeader(401)
            json.NewEncoder(w).Encode(errors.UnauthorizedError())
            return
        }

        accessToken := strings.Split(authHeader, " ")[1]

        rptResult, err := client.RetrospectToken(r.Context(), accessToken, clientId, clientSecret, realm)

        if err != nil {
            w.WriteHeader(400)
            json.NewEncoder(w).Encode(errors.BadRequestError(err.Error()))
            return
        }

        isTokenValid := *rptResult.Active

        if !isTokenValid {
            w.WriteHeader(401)
            json.NewEncoder(w).Encode(errors.UnauthorizedError())
            return
        }
        rpp, err := client.GetRequestingPartyPermissions(
            context.Background(),
            accessToken,
            realm,
            gocloak.RequestingPartyTokenOptions{
                Audience: gocloak.StringP(clientId),
                Permissions: &[]string{
                    "cluster#nerc-ocp-prod",
                    "namespace#all namespaces",
                },
            },
        )
        if err != nil {
            w.WriteHeader(401)
            json.NewEncoder(w).Encode(errors.UnauthorizedError())
            return
        }
        out, err := json.Marshal(*rpp)
        if err != nil {
            w.WriteHeader(400)
            json.NewEncoder(w).Encode(errors.BadRequestError(err.Error()))
            return
        }
        fmt.Print(string(out))

        // Our middleware logic goes here...
        next.ServeHTTP(w, r)
    })
}
EOF
echo DONE

Create a Prometheus service

In [ ]:
cat <<'EOF' > src/services/promService.go
// Thanks to okemechris on GitHub for the sample code. 
// See: https://github.com/okemechris/simplego-api/tree/main

package services

import (
    "encoding/json"
    "net/http"
    "github.com/OCP-on-NERC/prom-keycloak-proxy/src/errors"
)

func PromQuery(w http.ResponseWriter, r *http.Request) {
    data := new(errors.HttpError)
    json.NewEncoder(w).Encode(&data)
}
EOF
echo DONE

Create a controllers package and controller interface

In [ ]:
install -d src/controllers/

cat <<'EOF' > src/controllers/controller.go
// Thanks to okemechris on GitHub for the sample code. 
// See: https://github.com/okemechris/simplego-api/tree/main

package controllers

import (
    "github.com/gorilla/mux"
    "github.com/Nerzal/gocloak/v13"
)

type Controller interface {
    RegisterRoutes(client *gocloak.GoCloak, router *mux.Router)
}
EOF
echo DONE

Create a Prometheus Controller to define Prometheus Routes

In [ ]:
cat <<'EOF' > src/controllers/promController.go
// Thanks to okemechris on GitHub for the sample code. 
// See: https://github.com/okemechris/simplego-api/tree/main

package controllers
import (
    "net/http"
    "github.com/OCP-on-NERC/prom-keycloak-proxy/src/services"
    "github.com/gorilla/mux"
    "github.com/Nerzal/gocloak/v13"
)
type PromController struct {}
func (t PromController) RegisterRoutes(client *gocloak.GoCloak, router *mux.Router) {
    router.Handle("/api/v1/query",   services.Protect(client, http.HandlerFunc(services.PromQuery))).Methods("GET")
}
EOF
echo DONE

### Create a main package

In [ ]:
install -d src/

cat <<'EOF' > src/main.go
// Thanks to okemechris on GitHub for the sample code. 
// See: https://github.com/okemechris/simplego-api/tree/main

package main
import (
    "github.com/gorilla/mux"
    "github.com/Nerzal/gocloak/v13"
    "log"
    "net/http"
    controllers "github.com/OCP-on-NERC/prom-keycloak-proxy/src/controllers"
    services "github.com/OCP-on-NERC/prom-keycloak-proxy/src/services"
)
func main() {
    run()
}
func run() {
    client := services.InitializeOauthServer()
    router := mux.NewRouter().StrictSlash(true)
    router.Use(commonMiddleware)
    registerRoutes(client, router)
    log.Fatal(http.ListenAndServe(":8081", router))
}
func registerRoutes (client *gocloak.GoCloak, router *mux.Router){
    registerControllerRoutes(client, controllers.PromController{}, router)
}
func registerControllerRoutes(client *gocloak.GoCloak, controller controllers.Controller, router *mux.Router) {
    controller.RegisterRoutes(client, router)
}
func commonMiddleware(next http.Handler) http.Handler {
    return http.HandlerFunc(func(w http.ResponseWriter, r *http.Request) {
        w.Header().Add("Content-Type", "application/json")
        next.ServeHTTP(w, r)
    })
}
EOF
echo DONE